In [13]:
import blink.main_dense as main_dense

In [14]:
import os, json, glob2, pickle

In [15]:
from tqdm import tqdm 

In [16]:
import blink.main_dense as main_dense
import argparse

models_path = "/mnt/BIG-HDD-STORAGE/ebi/arxiv/processed/Models/models/" # the path where you stored the BLINK models

config = {
    "test_entities": None,
    "test_mentions": None,
    "interactive": False,
    "top_k": 5,
    "biencoder_model": models_path+"biencoder_wiki_large.bin",
    "biencoder_config": models_path+"biencoder_wiki_large.json",
    "entity_catalogue": models_path+"cs_related_wiki_and_pwc_entities.jsonl",
    "entity_encoding": models_path+"cs_related_wiki_and_pwc_entities.t7",
    "crossencoder_model": models_path+"crossencoder_wiki_large.bin",
    "crossencoder_config": models_path+"crossencoder_wiki_large.json",
    "fast": False, # set this to be true if speed is a concern
    "output_path": "logs/", # logging directory
    "faiss_index": "flat",
    "index_path": models_path+"faiss_flat_index_cs_related_wiki_and_pwc_entities.pkl"
}

args = argparse.Namespace(**config)

models = main_dense.load_models(args, logger=None)

In [17]:
base_dir = "/mnt/BIG-HDD-STORAGE/ebi/arxiv/processed"
triples_dir = f"{base_dir}/triples"
aug_triples_dir = f"{base_dir}/aug_triples_blink"

In [18]:
def process_triple(triple_file_dict):
    data_to_link = []
    for sub_section_index, sub_section in enumerate(triple_file_dict):
        if "entities_info" in sub_section:
            for entity_info in sub_section["entities_info"]:
                if entity_info["type"] not in ["Reference", "Code", "Number", "Ost"]:
                    bounds = sorted(
                        entity_info["bounds"],
                        key=lambda item: item[1] - item[0],
                        reverse=True,
                    )
                    proper_bound = None
                    for bound in bounds:
                        if (bound[1] - bound[0]) < 32:
                            proper_bound = bound
                            break
                    if proper_bound:
                        left_context = " ".join(
                            sub_section["tokens"][: proper_bound[0]]
                        )
                        mention = " ".join(
                            sub_section["tokens"][proper_bound[0] : proper_bound[1]]
                        )
                        right_context = " ".join(
                            sub_section["tokens"][proper_bound[1] :]
                        )
                        data_to_link.append(
                            {
                                "id": f'{sub_section_index}-{entity_info["key"]}',
                                "label": "unknown",
                                "label_id": -1,
                                "context_left": left_context.lower(),
                                "mention": mention.lower(),
                                "context_right": right_context.lower(),
                            }
                        )
    return data_to_link

In [19]:
def augment_triple(triple_file_dict, nel_result, data_to_link):
    for sub_section_index, sub_section in enumerate(triple_file_dict):
        if "entities_info" in sub_section:
            for entity_info in sub_section["entities_info"]:
                entity_id = f'{sub_section_index}-{entity_info["key"]}'
                found_indexes = [
                    i
                    for i in range(len(data_to_link))
                    if data_to_link[i]["id"] == entity_id
                ]
                if len(found_indexes) > 0:
                    found_index = found_indexes[0]
    return triple_file_dict

In [20]:
triple_files = glob2.glob(f"{triples_dir}/*.json")
pbar = tqdm(triple_files)
start = time.time()
for paper_counter, triple_file in enumerate(pbar):
    head, tail = os.path.split(triple_file)
    triple_file_dict = json.load(open(triple_file))
    data_to_link = process_triple(triple_file_dict)
    _, _, _, _, _, predictions, scores, = main_dense.run(args, None, *models, test_data=data_to_link)
    aug_triple_file_dict = augment_triple(
        triple_file_dict,
        {"scores": scores, "predictions": predictions},
        data_to_link,
    )
    save_dir = os.path.join(aug_triples_dir, tail)
    with open(save_dir, "w") as f_handler:
        json.dump(aug_triple_file_dict, f_handler)
    if (paper_counter == 9)
        break
end = time.time()
print(end - start)

  0%|                                                                                                                                                                                                                                                          | 0/211241 [00:00<?, ?it/s]

input_ids [101, 2073, 1038, 2003, 1996, 2561, 5166, 2408, 2608, 1010, 1047, 2003, 1996, 1, 2561, 2193, 1997, 2608, 2, 1998, 1056, 2003, 3091, 1997, 1996, 2051, 9154, 1012, 2023, 9038, 11302, 102]
max_seq_length 32
input_ids [101, 2516, 1024, 1996, 23807, 15873, 2791, 1997, 2358, 11663, 20875, 19088, 2000, 1, 6143, 16924, 2, 1012, 10061, 1024, 12774, 2011, 3171, 5097, 2107, 2004, 16755, 2121, 3001, 1010, 2057, 102]
max_seq_length 32
input_ids [101, 1010, 1047, 2003, 1996, 2561, 2193, 1997, 2608, 1998, 1056, 2003, 1, 3091, 1997, 1996, 2051, 9154, 2, 1012, 2023, 9038, 11302, 4324, 2104, 15275, 19293, 16924, 5656, 1997, 2608, 102]
max_seq_length 32
input_ids [101, 2433, 1037, 10594, 14442, 1012, 1996, 2896, 5391, 3084, 2224, 1997, 1037, 3722, 1, 16924, 5656, 2, 1010, 1996, 2168, 2005, 2035, 2093, 13792, 1010, 21336, 1037, 5391, 1997, 1179, 102]
max_seq_length 32
input_ids [101, 2000, 2433, 1037, 10594, 14442, 1012, 1996, 2896, 5391, 3084, 2224, 1997, 1037, 1, 3722, 2, 16924, 5656, 1010, 19


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00,  9.08it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71/71 [00:00<00:00, 256.93it/s]

input_ids [101, 2073, 1038, 2003, 1996, 2561, 5166, 2408, 2608, 1010, 1047, 2003, 1996, 1, 2561, 2193, 1997, 2608, 2, 1998, 1056, 2003, 3091, 1997, 1996, 2051, 9154, 1012, 2023, 9038, 11302, 102]
max_seq_length 32
input_ids [101, 2516, 1024, 1996, 23807, 15873, 2791, 1997, 2358, 11663, 20875, 19088, 2000, 1, 6143, 16924, 2, 1012, 10061, 1024, 12774, 2011, 3171, 5097, 2107, 2004, 16755, 2121, 3001, 1010, 2057, 102]
max_seq_length 32
input_ids [101, 1010, 1047, 2003, 1996, 2561, 2193, 1997, 2608, 1998, 1056, 2003, 1, 3091, 1997, 1996, 2051, 9154, 2, 1012, 2023, 9038, 11302, 4324, 2104, 15275, 19293, 16924, 5656, 1997, 2608, 102]
max_seq_length 32
input_ids [101, 2433, 1037, 10594, 14442, 1012, 1996, 2896, 5391, 3084, 2224, 1997, 1037, 3722, 1, 16924, 5656, 2, 1010, 1996, 2168, 2005, 2035, 2093, 13792, 1010, 21336, 1037, 5391, 1997, 1179, 102]
max_seq_length 32
input_ids [101, 2000, 2433, 1037, 10594, 14442, 1012, 1996, 2896, 5391, 3084, 2224, 1997, 1037, 1, 3722, 2, 16924, 5656, 1010, 19


Evaluation:  73%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                              | 52/71 [00:05<00:01, 10.30it/s]


  0%|                                                                                                                                                                                                                                                          | 0/211241 [00:10<?, ?it/s]

['K', 'total number of arms']  ->  Quantities of information  ->  -10.478192
['Strategic Manipulation', 'strategic manipulations']  ->  Strategic management  ->  -10.272943
['T', 'length of the time horizon']  ->  Time complexity  ->  -8.324804
['manipulation strategy']  ->  Strategy (game theory)  ->  -8.888692
['simple']  ->  Simple function  ->  -6.473478
['bandit algorithms']  ->  Sorting algorithm  ->  -8.966886
['B = o(T )']  ->  Branch and bound  ->  -9.925263
['bandits algorithms']  ->  Multi-armed bandit  ->  -6.0370765
['ε - Greedy']  ->  Greedy algorithm  ->  -5.5243564
['Thompson Sampling']  ->  Thompson sampling  ->  12.384498
['UCB']  ->  Branch and bound  ->  -9.4145
['Stochastic Bandits']  ->  Stochastic game  ->  -6.6240616
['economic applications']  ->  Applied economics  ->  -6.3351345
['recommender systems']  ->  Recommender system  ->  10.227101
['B']  ->  Barn (unit)  ->  -7.3812437
['Multi - armed bandits', 'MAB']  ->  Multi-armed bandit  ->  5.422364
['Amazon', 

In [6]:
def prepare_data_to_link(triple_file_dict, filename):
    data_to_link = []
    for sub_section in triple_file_dict:
        if "entities_info" in sub_section:
            for entity_info in sub_section["entities_info"]:
                if entity_info["type"] not in ["Reference", "Code", "Number", "Ost", "Feature"]:
                    bounds = sorted(entity_info['bounds'], key=lambda item: item[1] - item[0] ,reverse=True)
                    bound = bounds[0]
                    data_to_link.append({
                        'id': f'{filename}-{entity_info["key"]}',
                        'label': 'unknown',
                        'label_id': -1,
                        'context_left': ' '.join(sub_section["tokens"][:bound[0]]).lower(),
                        'mention': ' '.join(sub_section["tokens"][bound[0]:bound[1]]).lower(),
                        'context_right': ' '.join(sub_section["tokens"][bound[1]:]).lower(),
                    })
    _, _, _, _, _, predictions, scores, = main_dense.run(args, None, *models, test_data=data_to_link)
    for dtl, prediction, score in zip(data_to_link, predictions, scores):
        if(score[0] > 0):
            print(dtl['mention'], prediction[0], score[0])

In [7]:
import time

In [9]:
triple_files = glob2.glob(f"{triples_dir}/*.json")
pbar = tqdm(triple_files)
start = time.time()
for counter, triple_file in enumerate(pbar):
    head, tail = os.path.split(triple_file)
    with open(triple_file) as file_handler:
        triple_file_dict = json.load(file_handler)
    prepare_data_to_link(triple_file_dict, tail)
    if counter == 10:
        break
end = time.time()
print(end - start)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 11.90it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 65/65 [00:00<00:00, 233.32it/s]


  0%|                                                                                                | 1/211241 [00:11<646:39:14, 11.02s/it]

thompson sampling Thompson sampling 12.384498
recommender systems Recommender system 10.227101
multi - armed bandits Multi-armed bandit 5.422364
amazon marketplace Amazon Marketplace 11.996152
restaurants Restaurant 11.597531
digital economy Digital economy 8.752863
recommender systems Recommender system 8.897625
search engines Search engine (computing) 3.2318892
online advertising Online advertising 10.1705885
service quality Service quality 10.724719
yelp Yelp 10.2075405
regret Regret 0.62473506
recommender systems Recommender system 10.332027
auctions Auction 8.028332



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 11.73it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 56/56 [00:00<00:00, 281.23it/s]


  0%|                                                                                                | 2/211241 [00:20<603:11:49, 10.28s/it]

...


  0%|                                                                                               | 10/211241 [01:50<719:57:54, 12.27s/it]

division algebras Division algebra 0.07310968
spectral efficiency Spectral efficiency 10.687772
division algebras Division algebra 0.3143161
spectral efficiency Spectral efficiency 10.687772
mc - cdma Multi-carrier code-division multiple access 9.864821
minimum mean square error Minimum mean square error 10.695292
m ultiple - input multiple - output MIMO 5.8448567
data rates Bit rate 6.5784163
multiuser detection Multiuser detection 11.886761
hopfield neural network Hopfield network 6.0124483
cdma Code-division multiple access 6.61821
image restoration Image restoration 9.548999
spectral efficiency Spectral efficiency 10.687772
antennas Antenna (radio) 6.7620044
ieee 802.11n IEEE 802.11n-2009 9.893772



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 11.89it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 65/65 [00:00<00:00, 218.87it/s]



  0%|                                                                                               | 10/211241 [02:02<716:12:05, 12.21s/it]

neural networks Neural network 4.120176
encoder Encoder 8.472188
generative adversarial networks Generative adversarial network 10.641603
social networks Social network 6.4440527
generative adversarial network - based methods Generative adversarial network 5.612352
encoder Encoder 7.651771
effectiveness Effectiveness 5.438431
word embeddings Word embedding 5.9347057
part - of - speech Part of speech 11.306923
support vector machine Support-vector machine 12.368055
logistic regression Logistic regression 11.527652
random forest Random forest 10.39833
classifier Classifier (linguistics) 1.8419323
deep learning based methods Deep learning 1.63243
performance Performance 1.8687394
deep learning based methods Deep learning 1.63243
generative approach Generative model 0.04395907
performance Performance 1.8687394
fever Fever 0.14037563
textual entailment Textual entailment 9.415311
effectiveness Effectiveness 7.01189
122.05610013008118
